In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 64;
nsm = 64;
nips = 20;
nrps = 1;
nb = 32;
measperiod = 1;
nt = 32
nh = 13

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 6,
                        help = 'number of threads to use')

    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--randominit', default = False, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = True, action = 'store_true',
                        help = '''initialise all the temperature with the maximally magnetised GS''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--checkssf', default = True, action = 'store_true',
                        help = 'activate to debug ssf')
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.01,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [-1.0, 1.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "tests/testJ1h", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [5]:
%lprun -f dw.mcs_swaps meanstat, swapst, swapsh, failedupdatesth, failedupdates = callRunBasis() 

Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
Number of temperatures:  32
Temperatures: [1.00000000e-02 1.32396300e-02 1.75287801e-02 2.32074562e-02
 3.07258133e-02 4.06798398e-02 5.38586025e-02 7.13067968e-02
 9.44075602e-02 1.24992116e-01 1.65484937e-01 2.19095932e-01
 2.90074907e-01 3.84048443e-01 5.08465926e-01 6.73190071e-01
 8.91278743e-01 1.18002007e+00 1.56230291e+00 2.06843124e+00
 2.73852642e+00 3.62570765e+00 4.80030276e+00 6.35542322e+00
 8.41434516e+00 1.11402816e+01 1.47493206e+01 1.95275547e+01
 2.58537598e+01 3.42294213e+01 4.53184872e+01 6.00000000e+01]
Number of magnetic fields:  13
Magnetic fields:  [-1.         -0.83333333 -0.66666667 -0.5        -0.33333333 -0.16666667
  0.          0.16666667  0.33333333  0.5         0.66666667  0.83333333
  1.        ]
Fully random initialisation =  False
Identical initialisation =  False
Magnetisation initialisation =  True
Initialisation type:  magninit
Magn

Time for mcsevolve = 0.45647405460476875
Time for tempering = 0.004900399595499039
Time for mapping to spins + computing statistics= 0.054211199283599854
Energies size:  (32, 13)
Time for all measurements steps =  33.25447678565979
[-1296. -1224. -1152. -1080. -1008.  -936.  -864.  -936. -1008. -1080.
 -1152. -1224. -1296.]
RunBasis: Away from gs at t index  5  and h index  7
   new_en_states[t,h] =  -935.3333333333333
   ref_en_states[h] =  -935.9999999999999
RunBasis: Away from gs at t index  6  and h index  5
   new_en_states[t,h] =  -933.6666666666667
   ref_en_states[h] =  -936.0
RunBasis: Away from gs at t index  6  and h index  7
   new_en_states[t,h] =  -935.3333333333333
   ref_en_states[h] =  -935.9999999999999
RunBasis: Away from gs at t index  7  and h index  5
   new_en_states[t,h] =  -927.6666666666667
   ref_en_states[h] =  -936.0
RunBasis: Away from gs at t index  7  and h index  7
   new_en_states[t,h] =  -928.0
   ref_en_states[h] =  -935.9999999999999
RunBasis: Away 

Job done


In [6]:
print(failedupdatesth/(nst*nips))

[[1.         1.         1.         1.         1.         1.
  0.55368622 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55443854 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55285012 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55403344 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         0.99997589
  0.55293029 0.99997046 1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         0.99962445
  0.55292908 0.99966122 1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         0.99999156 0.99745069
  0.5541745  0.99744948 0.99999518 1.         1.         1.
  1.        ]
 [1.         1.         1.         1. 

In [7]:
print(failedupdates/(nsm*nips*measperiod))

[[1.         1.         1.         1.         1.         1.
  0.55410397 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55315152 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55412326 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         1.
  0.55440357 1.         1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         0.99997709
  0.55364161 0.99997348 1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         1.         0.9996365
  0.55379835 0.99963891 1.         1.         1.         1.
  1.        ]
 [1.         1.         1.         1.         0.99999277 0.99743562
  0.55261803 0.99742417 0.99999699 1.         1.         1.
  1.        ]
 [1.         1.         1.         1.  

In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[1.         1.         1.         0.98557692 0.92307692 0.875
 0.82692308 0.70192308 0.65384615 0.47115385 0.32692308 0.09615385
 0.09615385 0.10576923 0.08653846 0.04326923 0.01442308 0.00480769
 0.00480769 0.00961538 0.05288462 0.04326923 0.13461538 0.20192308
 0.36057692 0.49519231 0.55288462 0.61057692 0.75961538 0.75480769
 0.875      0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.65820312 0.63671875 0.58789062 0.52734375 0.4609375  0.296875
 0.31054688 0.4375     0.54882812 0.58398438 0.60546875 0.66210938
 0.        ]
